In [1]:
import numpy as np
import pandas as pd
import h5py
import pysam
import os
from modisco.visualization import viz_sequence
from modisco import util
from matplotlib import pyplot as plt
import pybedtools

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [2]:
#dttype = 'atac'
#mode = 'counts'

#modisco_path = '/oak/stanford/groups/akundaje/projects/chrombpnet_paper_new/ATAC_PE/K562/uncorrected_model_08.22.2022_filters_512_dil_8/SIGNAL/modisco_crop_500/modisco_results_allChroms_'+mode+'.hdf5'
#ppm_dir = '/mnt/lab_data3/anusri/chrombpnet/results/chrombpnet/ATAC_PE/K562/uncorrected_model_08.22.2022_filters_512_dil_8/'
#tomtom = pd.read_csv("/oak/stanford/groups/akundaje/projects/chrombpnet_paper_new/ATAC_PE/K562/uncorrected_model_08.22.2022_filters_512_dil_8/SIGNAL/modisco_crop_500/" + mode + ".tomtom.tsv", sep="\t")

dttype = 'ATAC'
mode = "counts"
celltype="K562"
modisco_path = '/oak/stanford/groups/akundaje/projects/chrombpnet_paper_new/modisco_jun_30/modisco/'+dttype+'/'+celltype+'/modiscolite_crop_500_1M_seqs/modisco_results_allChroms_'+mode+'.hdf5'
ppm_dir = 'subfigs/'+dttype+'/'+celltype+'/'
tomtom = pd.read_csv('/oak/stanford/groups/akundaje/projects/chrombpnet_paper_new/modisco_jun_30/modisco/'+dttype+'/'+celltype+'/modiscolite_crop_500_1M_seqs/'+ mode + ".tomtom.tsv", sep="\t")




background=[0.25, 0.25, 0.25, 0.25]

In [3]:
def trim_motif_new(cwm, motif, trim_threshold=0.20):
    """
    Given the PFM and motif (both L x 4 arrays) (the motif could be the
    PFM itself), trims `motif` by cutting off flanks of low information
    content in `pfm`. `min_ic` is the minimum required information
    content. If specified this trimmed motif will be extended on either
    side by `pad` bases.
    If no base passes the `min_ic` threshold, then no trimming is done.
    """
    
    score = np.sum(np.abs(cwm), axis=1)
    trim_thresh = np.max(score) * trim_threshold  # Cut off anything less than 30% of max score
    pass_inds = np.where(score >= trim_thresh)[0]
    trimmed = motif[np.min(pass_inds): np.max(pass_inds) + 1]
 
    if not trimmed.size:
        return motif
    
    return trimmed

def import_tfmodisco_motifs(tfm_results_path, trim=True, only_pos=True):
    """
    Imports the PFMs to into a dictionary, mapping `(x, y)` to the PFM,
    where `x` is the metacluster index and `y` is the pattern index.
    Arguments:
        `tfm_results_path`: path to HDF5 containing TF-MoDISco results
        `out_dir`: where to save motifs
        `trim`: if True, trim the motif flanks based on information content
        `only_pos`: if True, only return motifs with positive contributions
    Returns the dictionary of PFMs.
    """ 
    pfms = {}
    with h5py.File(tfm_results_path, "r") as f:
        metaclusters = f["metacluster_idx_to_submetacluster_results"]
        num_metaclusters = len(metaclusters.keys())
        for metacluster_i, metacluster_key in enumerate(metaclusters.keys()):
            metacluster = metaclusters[metacluster_key]
            if "patterns" not in metacluster["seqlets_to_patterns_result"]:
                continue
            patterns = metacluster["seqlets_to_patterns_result"]["patterns"]
            num_patterns = len(patterns["all_pattern_names"][:])
            for pattern_i, pattern_name in enumerate(patterns["all_pattern_names"][:]):
#                pattern_name = pattern_name.decode()
                pattern_name = pattern_name

                pattern = patterns[pattern_name]
                pfm = pattern["sequence"]["fwd"][:]
                cwm = pattern["task0_contrib_scores"]["fwd"][:]
                
                # Check that the contribution scores are overall positive
                if only_pos and np.sum(cwm) < 0:
                    continue
                    
                if trim:
                    pfm = trim_motif_new(cwm, cwm)
                else:
                    pfm = cwm
                    
                pfms["%d_%d" % (metacluster_i,pattern_i)] = pfm
    return pfms

In [4]:
pfms = import_tfmodisco_motifs(modisco_path, trim=False)

In [5]:
for key in pfms:
    f = open(os.path.join(ppm_dir,mode+"_"+key+".pfm"),"w")
    #print(pfms[key])
    np.savetxt(f, pfms[key], fmt='%f')
    f.close()
    

In [6]:
tomtom.head()

,Pattern,Num_Seqlets,Match_1,q-value,Match_2,q-value.1,Match_3,q-value.2,Match_4,q-value.3,Match_5,q-value.4,Match_6,q-value.5,Match_7,q-value.6,Match_8,q-value.7,Match_9,q-value.8,Match_10,q-value.9
0,metacluster_0.pattern_0,33393,GATA3_GATA_1,2.863010e-01,GATA3_GATA_2,2.863010e-01,GATA3_MA0037.3,2.863010e-01,GATA4_GATA_1,2.863010e-01,GATA5_GATA_1,2.863010e-01,GATA5_MA0766.1,0.286301,GATA4_HUMAN.H11MO.0.A,0.286301,GATA4_MOUSE.H11MO.0.A,0.286301,GATA6_MOUSE.H11MO.0.A,0.286301,GATA2_MA0036.3,0.286301
1,metacluster_0.pattern_1,30688,CTCF_MA0139.1,7.883100e-15,CTCF_HUMAN.H11MO.0.A,3.590460e-12,CTCF_MOUSE.H11MO.0.A,3.590460e-12,CTCFL_HUMAN.H11MO.0.A,1.155980e-07,CTCFL_MOUSE.H11MO.0.A,2.794360e-07,CTCF_C2H2_1,0.000001,CTCFL_MA1102.1,0.000009,ZIC3_HUMAN.H11MO.0.B,0.165402,ZIC3_MOUSE.H11MO.0.A,0.165402,ZIC2_MOUSE.H11MO.0.C,0.169989
2,metacluster_0.pattern_2,23865,KLF3_HUMAN.H11MO.0.B,1.059640e-05,KLF3_MOUSE.H11MO.0.A,1.059640e-05,KLF12_HUMAN.H11MO.0.C,1.059640e-05,SP3_HUMAN.H11MO.0.B,1.059640e-05,SP3_MOUSE.H11MO.0.B,1.059640e-05,SP1_MA0079.3,0.000053,SP1_HUMAN.H11MO.0.A,0.000211,SP1_MOUSE.H11MO.0.A,0.000211,KLF5_MA0599.1,0.000336,SP4_HUMAN.H11MO.0.A,0.000520
3,metacluster_0.pattern_3,12213,BACH2_HUMAN.H11MO.0.A,1.039430e-05,BACH2_MOUSE.H11MO.0.A,1.039430e-05,BACH1_HUMAN.H11MO.0.A,1.385900e-05,Bach1+Mafk_MA0591.1,2.820880e-05,BACH2_MA1101.1,3.228190e-05,NFE2_HUMAN.H11MO.0.A,0.000035,NF2L2_HUMAN.H11MO.0.A,0.000038,MAF+NFE2_MA0501.1,0.000040,NF2L2_MOUSE.H11MO.0.A,0.000041,NFE2_MOUSE.H11MO.0.A,0.000041
4,metacluster_0.pattern_4,9525,ELK1_ETS_1,3.838900e-04,ELK3_ETS_1,3.838900e-04,ELK3_MA0759.1,3.838900e-04,ELK4_MA0076.2,3.838900e-04,Gabpa_MA0062.2,3.838900e-04,EHF_ETS_1,0.000384,EHF_MA0598.2,0.000384,ELK4_HUMAN.H11MO.0.A,0.000384,ETS1_ETS_4,0.000676,ELK1_ETS_2,0.000676


In [7]:

tomtom["Pattern"] = tomtom["Pattern"].str.replace("metacluster_","").str.replace(".pattern_","_")

/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [8]:
tomtom.head()

,Pattern,Num_Seqlets,Match_1,q-value,Match_2,q-value.1,Match_3,q-value.2,Match_4,q-value.3,Match_5,q-value.4,Match_6,q-value.5,Match_7,q-value.6,Match_8,q-value.7,Match_9,q-value.8,Match_10,q-value.9
0,0_0,33393,GATA3_GATA_1,2.863010e-01,GATA3_GATA_2,2.863010e-01,GATA3_MA0037.3,2.863010e-01,GATA4_GATA_1,2.863010e-01,GATA5_GATA_1,2.863010e-01,GATA5_MA0766.1,0.286301,GATA4_HUMAN.H11MO.0.A,0.286301,GATA4_MOUSE.H11MO.0.A,0.286301,GATA6_MOUSE.H11MO.0.A,0.286301,GATA2_MA0036.3,0.286301
1,0_1,30688,CTCF_MA0139.1,7.883100e-15,CTCF_HUMAN.H11MO.0.A,3.590460e-12,CTCF_MOUSE.H11MO.0.A,3.590460e-12,CTCFL_HUMAN.H11MO.0.A,1.155980e-07,CTCFL_MOUSE.H11MO.0.A,2.794360e-07,CTCF_C2H2_1,0.000001,CTCFL_MA1102.1,0.000009,ZIC3_HUMAN.H11MO.0.B,0.165402,ZIC3_MOUSE.H11MO.0.A,0.165402,ZIC2_MOUSE.H11MO.0.C,0.169989
2,0_2,23865,KLF3_HUMAN.H11MO.0.B,1.059640e-05,KLF3_MOUSE.H11MO.0.A,1.059640e-05,KLF12_HUMAN.H11MO.0.C,1.059640e-05,SP3_HUMAN.H11MO.0.B,1.059640e-05,SP3_MOUSE.H11MO.0.B,1.059640e-05,SP1_MA0079.3,0.000053,SP1_HUMAN.H11MO.0.A,0.000211,SP1_MOUSE.H11MO.0.A,0.000211,KLF5_MA0599.1,0.000336,SP4_HUMAN.H11MO.0.A,0.000520
3,0_3,12213,BACH2_HUMAN.H11MO.0.A,1.039430e-05,BACH2_MOUSE.H11MO.0.A,1.039430e-05,BACH1_HUMAN.H11MO.0.A,1.385900e-05,Bach1+Mafk_MA0591.1,2.820880e-05,BACH2_MA1101.1,3.228190e-05,NFE2_HUMAN.H11MO.0.A,0.000035,NF2L2_HUMAN.H11MO.0.A,0.000038,MAF+NFE2_MA0501.1,0.000040,NF2L2_MOUSE.H11MO.0.A,0.000041,NFE2_MOUSE.H11MO.0.A,0.000041
4,0_4,9525,ELK1_ETS_1,3.838900e-04,ELK3_ETS_1,3.838900e-04,ELK3_MA0759.1,3.838900e-04,ELK4_MA0076.2,3.838900e-04,Gabpa_MA0062.2,3.838900e-04,EHF_ETS_1,0.000384,EHF_MA0598.2,0.000384,ELK4_HUMAN.H11MO.0.A,0.000384,ETS1_ETS_4,0.000676,ELK1_ETS_2,0.000676


In [9]:
tomtom[["Pattern","Num_Seqlets"]].to_csv(os.path.join(ppm_dir,mode+"_counts.csv"),sep=",",index=False, header=False)